# count min sketch source :
https://github.com/rafacarrascosa/countminsketch

In [3]:
import pip
pip.main(['install', "countminsketch"])

0

In [8]:
#from count_min_sketch import CountMinSketch
#from countminsketch import CountMinSketch
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import math

In [9]:
p= 10000
w = 4

#column_names = ['A','B','C','D','E','F','G','H','I','J']
#data_frame = pd.read_fwf("F:\\IIT HYDERABAD NOTES\\Topics in DBMS\\yeast.data",
#                         sep = '\t',names = column_names)
#print data_frame.head()

In [4]:
"""data_frame['A'] = data_frame['A'].astype('category')
data_frame['J'] = data_frame['J'].astype('category')
cat_cols = ['A','J']

data_frame[cat_cols] = data_frame[cat_cols].apply(lambda x : x.cat.codes)
print data_frame.head()
print(data_frame.shape)

#data_list = data_frame.values.tolist()
"""

"data_frame['A'] = data_frame['A'].astype('category')\ndata_frame['J'] = data_frame['J'].astype('category')\ncat_cols = ['A','J']\n\ndata_frame[cat_cols] = data_frame[cat_cols].apply(lambda x : x.cat.codes)\nprint data_frame.head()\nprint(data_frame.shape)\n\n#data_list = data_frame.values.tolist()\n"

In [10]:
waveform_data = pd.read_csv("waveform.data",header = None)

count = 0

#for i in range(0,len(waveform_data)):
#    waveform_data[i] = tuple(waveform_data[i])
    

print(waveform_data.head())

     0     1     2     3     4     5     6     7     8     9  ...    12    13  \
0 -1.23 -1.56 -1.75 -0.28  0.60  2.22  0.85  0.21 -0.20  0.89 ...  2.89  7.75   
1 -0.69  2.43  0.61  2.08  2.30  3.25  5.52  4.55  2.97  2.22 ...  1.24  1.89   
2 -0.12 -0.94  1.29  2.59  2.42  3.55  4.94  3.25  1.90  2.07 ...  2.50  0.12   
3  0.86  0.29  2.19 -0.02  1.13  2.51  2.37  5.45  5.45  4.84 ...  2.58  1.40   
4  1.16  0.37  0.40 -0.59  2.66  1.00  2.69  4.06  5.34  3.53 ...  4.30  1.84   

     14    15    16    17    18    19    20  21  
0  4.59  3.15  5.12  3.32  1.20  0.24 -0.56   2  
1  1.88 -1.34  0.83  1.41  1.78  0.60  2.42   1  
2  1.41  2.78  0.64  0.62 -0.01 -0.79 -0.12   0  
3  1.24  1.41  1.07 -1.43  2.84 -1.18  1.12   1  
4  1.73  0.21 -0.18  0.13 -0.21 -0.80 -0.68   1  

[5 rows x 22 columns]


In [48]:
a = CountMinSketch(p,w)

In [6]:
#print(data_frame.ix[1].values.tolist())
#a[1] = data_frame.ix[1].values.tolist()
"""a[1] = 2
a[1] += 1
for i in a.get_columns(1):
    print i

print(a.query(1))
"""

#a.add(tuple([1,2,3]))
#print a[tuple([1,2,3])]
#print a[tuple([1,2,3]]
#print(a.tables)
#p = [4,5,6]
#p = np.vectorize(p)
#print(p[0])




'a[1] = 2\na[1] += 1\nfor i in a.get_columns(1):\n    print i\n\nprint(a.query(1))\n'

In [11]:
seed_np = np.random.seed(1)
s = 1000.0                                                      #sample size
sqrt_s = np.sqrt(s)

f = np.random.uniform(1.0/sqrt_s,1.0 - (1.0/sqrt_s))            #locality parameter

print(f)

0.42226999391886616


In [12]:
d = waveform_data.shape[1]                                         #dimension of the dataset

random_vector = np.random.uniform(0,f,d)*10
print(random_vector)

[  3.04171419e+00   4.82970534e-04   1.27665974e+00   6.19706091e-01
   3.89918179e-01   7.86520983e-01   1.45919926e+00   1.67542999e+00
   2.27526139e+00   1.77013265e+00   2.89347634e+00   8.63340503e-01
   3.70802645e+00   1.15649588e-01   2.83118311e+00   1.76215296e+00
   2.35917950e+00   5.92811917e-01   8.36523146e-01   3.38130404e+00
   4.08867810e+00   1.32349626e+00]


In [13]:
sample_data = waveform_data.sample(n = s)
print(sample_data.shape)
base = np.maximum(2,1.0/f)
print(base)

(1000, 22)
2.36815311152


/usr/lib/python3/dist-packages/pandas/core/generic.py:2258: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  locs = rs.choice(axis_length, size=n, replace=replace, p=weights)


In [14]:
#randomly select 'r' columns from dataset
np.random.seed(1)
r = int(np.random.uniform(1.0+(0.5*math.log(s,base)),math.log(s,base)))
waveform_randomly = waveform_data.sample(n=r,axis = 1)
print(waveform_randomly.head())


     13    14    16    1     20    19
0  7.75  4.59  5.12 -1.56 -0.56  0.24
1  1.89  1.88  0.83  2.43  2.42  0.60
2  0.12  1.41  0.64 -0.94 -0.12 -0.79
3  1.40  1.24  1.07  0.29  1.12 -1.18
4  1.84  1.73 -0.18  0.37 -0.68 -0.80


In [15]:
#sample 1000 data from randomly_waveform
waveform_randomly_train = waveform_randomly.sample(n=1000,axis = 0).values.tolist()
print(waveform_randomly_train[0:3])


#for i in range(0,len(waveform_data)):
#    waveform_randomly[i] = tuple(waveform_data[i])

[[1.06, 0.46, -0.07, 1.51, -0.11, 0.82], [1.85, 5.55, 3.63, -0.47, 0.36, 0.47], [3.76, 7.05, 3.59, -0.12, -1.26, 2.06]]


In [16]:
col_max = np.max(waveform_randomly_train,axis = 0)
print(col_max)

[ 7.75  8.72  6.74  3.83  3.1   4.28]


In [17]:
col_min = np.min(waveform_randomly_train, axis = 0)
print (col_min)

[-2.37 -2.56 -2.71 -2.96 -3.45 -3.25]


In [20]:
normalized_waveform = (waveform_randomly_train - col_min)/(col_max -  col_min)
print(normalized_waveform[0])

[ 0.33893281  0.2677305   0.27936508  0.65832106  0.50992366  0.54050465]
